# DS776 Course Setup

Welcome to DS776 Deep Learning! This notebook will set up your course environment.

**This notebook will:**
1. Install the `introdl` course package
2. Set up API key configuration
3. Verify everything is working

## ⚠️ Important Notes:
- Run this notebook ONCE at the beginning of the course
- Run all cells in order
- The setup takes about 1-2 minutes
- Workspace folders are created automatically when you import `introdl`

## Step 1: Install or Update Course Package

This uninstalls any existing `introdl` package and then installs a fresh version.

In [1]:
# Install the introdl package
import subprocess
import sys
from pathlib import Path

print("📦 Installing/updating introdl course package...")
print("This may take 1-2 minutes...\n")

# First, uninstall existing package if present
print("🔄 Checking for existing installation...")
result = subprocess.run(
    [sys.executable, "-m", "pip", "show", "introdl"],
    capture_output=True,
    text=True
)

if result.returncode == 0:
    print("   Found existing installation, uninstalling...")
    uninstall_result = subprocess.run(
        [sys.executable, "-m", "pip", "uninstall", "introdl", "-y", "--quiet"],
        capture_output=True,
        text=True
    )
    if uninstall_result.returncode == 0:
        print("   ✅ Previous version uninstalled")
    else:
        print("   ⚠️ Could not uninstall previous version")
else:
    print("   No existing installation found")

print("\n📦 Installing fresh package...")

# Get the path to the introdl package
course_tools_dir = Path.cwd().parent  # Assuming we're in Course_Tools
introdl_dir = course_tools_dir / "introdl"

if not introdl_dir.exists():
    # Try alternative path
    introdl_dir = Path.cwd() / "introdl"

if introdl_dir.exists():
    # Install the package (non-editable for CoCalc compatibility)
    result = subprocess.run(
        [sys.executable, "-m", "pip", "install", str(introdl_dir), "--quiet"],
        capture_output=True,
        text=True
    )
    
    if result.returncode == 0:
        print("✅ Package installed successfully!")
        print("   Note: Using standard install (not editable) for CoCalc compatibility")
    else:
        print("❌ Installation failed. Error:")
        print(result.stderr)
else:
    print("❌ Could not find introdl package directory")
    print(f"   Looked in: {introdl_dir}")

📦 Installing/updating introdl course package...
This may take 1-2 minutes...

🔄 Checking for existing installation...


   Found existing installation, uninstalling...


   ✅ Previous version uninstalled

📦 Installing fresh package...


✅ Package installed successfully!
   Note: Using standard install (not editable) for CoCalc compatibility


In [ ]:
import shutil
from pathlib import Path

print("🔑 Setting up API key configuration...\n")

# Determine the course root and workspace location
current_dir = Path.cwd()
if "Course_Tools" in str(current_dir):
    # We're in Lessons/Course_Tools
    course_root = current_dir.parent.parent
    course_tools_dir = current_dir
else:
    # Try to find the course root
    for parent in [current_dir] + list(current_dir.parents):
        if (parent / "Lessons").exists() and (parent / "Homework").exists():
            course_root = parent
            course_tools_dir = course_root / "Lessons" / "Course_Tools"
            break
    else:
        course_root = Path.home() / "DS776"
        course_tools_dir = course_root / "Lessons" / "Course_Tools"

# The config_paths_keys function will create home_workspace automatically
# We just need to ensure the api_keys.env file exists
home_workspace = course_root / "home_workspace"
home_workspace.mkdir(parents=True, exist_ok=True)

# Create api_keys.env in home_workspace if it doesn't exist
api_keys_dest = home_workspace / "api_keys.env"
api_keys_template = course_tools_dir / "api_keys.env"

if not api_keys_dest.exists():
    if api_keys_template.exists():
        # Copy the template
        shutil.copy2(api_keys_template, api_keys_dest)
        print(f"✅ Created API keys file: {api_keys_dest.relative_to(course_root)}")
    else:
        # Create a new template
        template_content = """# API Keys for DS776 Deep Learning Course
# Edit this file with your actual API keys
# This file is in home_workspace and will sync across compute servers

# OpenAI API Key (for GPT models)
OPENAI_API_KEY=your_openai_key_here

# Hugging Face Token (for model downloads)
HF_TOKEN=your_huggingface_token_here

# OpenRouter API Key (for various LLMs)
OPENROUTER_API_KEY=your_openrouter_key_here

# Google API Key (for Gemini models)
GOOGLE_API_KEY=your_google_key_here

# Groq API Key (for fast inference)
GROQ_API_KEY=your_groq_key_here
"""
        api_keys_dest.write_text(template_content)
        print(f"✅ Created API keys template: {api_keys_dest.relative_to(course_root)}")
    
    print("\n📝 TO ADD YOUR API KEYS:")
    print(f"   1. Open {api_keys_dest.relative_to(course_root)}")
    print("   2. Replace 'your_xxx_key_here' with your actual API keys")
    print("   3. Save the file")
    print("\n   Your keys will automatically sync across compute servers!")
else:
    print(f"✅ API keys file already exists: {api_keys_dest.relative_to(course_root)}")
    print("   (Your existing keys have been preserved)")

## Step 3: Verify Installation

Let's make sure everything is working correctly.

In [ ]:
print("🔍 Verifying installation...\n")

# Test 1: Import introdl package
try:
    import introdl
    print("✅ introdl package imported successfully")
    print(f"   Version: {getattr(introdl, '__version__', 'Unknown')}")
except ImportError as e:
    print(f"❌ Could not import introdl: {e}")

# Test 2: Test config_paths_keys (this will also create workspace folders)
print("\n🔧 Testing configuration and creating workspace...")
try:
    from introdl.utils import config_paths_keys
    import os
    
    # Set DS776_ROOT_DIR if we found the course root
    if 'course_root' in locals():
        os.environ['DS776_ROOT_DIR'] = str(course_root)
    
    # This will automatically create all necessary workspace folders
    paths = config_paths_keys()
    print("✅ Configuration loaded successfully")
    print("\n📁 Workspace paths configured:")
    print(f"   Data:   {paths['DATA_PATH']}")
    print(f"   Models: {paths['MODELS_PATH']}")
    print(f"   Cache:  {paths['CACHE_PATH']}")
except Exception as e:
    print(f"❌ Configuration error: {e}")

# Test 3: Check API keys file
print("\n🔑 API keys file:")
if 'api_keys_dest' in locals() and api_keys_dest.exists():
    print(f"   ✅ {api_keys_dest.relative_to(course_root)}")
else:
    print(f"   ✅ API keys file configured")

# Test 4: Check PyTorch
print("\n🔥 Checking PyTorch...")
try:
    import torch
    print(f"✅ PyTorch version: {torch.__version__}")
    if torch.cuda.is_available():
        print(f"   GPU available: {torch.cuda.get_device_name(0)}")
    else:
        print("   Running on CPU (GPU not available)")
except ImportError:
    print("❌ PyTorch not installed")

## 🎉 Setup Complete\!

Your DS776 course environment is now configured and ready to use.

### Next Steps:
1. **Add your API keys** to  if you haven't already
2. **Start with Lesson 01** notebooks
3. **Workspace folders** are created automatically when you run notebooks

### Need Help?
- **Storage issues?** Use the  notebook
- **Package issues?** Use the  notebook
- **API key issues?** Check that your keys are correctly entered in 

Good luck with the course\! 🚀